# Clean scraped Abu Dhabi GP tweets

In [ ]:
%%capture
# pandas now requires openpyxl 3.0.0
!pip install openpyxl==3.0.0

In [ ]:
import openpyxl
print(f"openpyxl version: {openpyxl.__version__}")

import pandas as pd
import numpy as np
import re 
import ast

openpyxl version: 3.0.0


In [ ]:
# Mount drive to access files
%%capture
from google.colab import drive
drive.mount('/content/drive/')
!ls "/content/drive/My Drive"

In [ ]:
race_df = pd.read_excel('drive/My Drive/data/abudhabigp_fullrace_english_deduped.xlsx')
print(race_df.shape)
race_df.head()

(252636, 5)


,date,timezone,tweet,language,hashtags
0,2021-12-12 12:59:59.999999,0,@MercedesAMGF1 My nerves are wrecking the anxi...,en,[]
1,2021-12-12 12:59:59.999999,0,Lets go! @LewisHamilton https://t.co/QiL9lWU6RQ,en,[]
2,2021-12-12 12:59:59.999999,0,Turn 1 anxiety #AbuDhabiGP,en,['abudhabigp']
3,2021-12-12 12:59:59.999999,0,@piersmorgan @LewisHamilton Real brits pick wh...,en,[]
4,2021-12-12 12:59:59.999999,0,@chris_tiffa @LewisHamilton Haha aye funny tha...,en,[]


In [ ]:
race_df['date'] = race_df['date'].round('1s')

In [ ]:
race_df['hashtags'] = [ast.literal_eval(x) for x in race_df['hashtags']]
assert all([type(x) == list for x in race_df['hashtags']])

## Clean tweets
- Keep top english word hashtags & top handles
- Remove junk tweets 
- 5 minute intervals for sampling and plotting

In [ ]:
# Datetimes
race_df['date'] = pd.to_datetime(race_df['date'], format = '%Y-%m-%d %H:%M:%S') 

# Create intervals 
race_df['interval'] = race_df['date'].dt.round('5min')
race_df['interval'] = [x.strftime('%H:%M') for x in race_df['interval']]

# Remove lengthy junk tweets
race_df['tweet_len'] = [len(x) for x in race_df['tweet']]
print(f"Long tweets:\n{race_df[race_df['tweet_len'] >= 512][['tweet', 'tweet_len']]}")

race_df = race_df[race_df['tweet_len'] < 512]
print(f"{len(race_df)} tweets left")

Long tweets:
                                                    tweet  tweet_len
43765   perez &gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;&gt;...        806
110993  @familysherrod @barryjohnreid21 @TheFrankmanMN...        733
252634 tweets left


In [ ]:
# id the remaining tweets
race_df = race_df.reset_index(drop=True).reset_index()
race_df = race_df.rename(columns = {'index': 'id'})

In [ ]:
# Check that index and id still tally
assert race_df.reset_index()[['index', 'id']].apply(lambda x: x['index'] - x['id'], axis=1).max() == 0

In [ ]:
# Inspect top 200 hashtags to identify top english words
top_200_hashtags = list(race_df['hashtags'].explode()\
                       .value_counts(ascending = False)\
                        .reset_index().iloc[0:200, 0])
# Inspect top 100 handles 
top_100_handles = list(pd.Series([re.findall('@[A-Za-z0-9]+', x) for x in race_df['tweet']])\
                  .explode().value_counts().reset_index().iloc[0:100, 0])

print(f"Top hashtags: {top_200_hashtags}\nTop handles: {top_100_handles}")

Top hashtags: ['abudhabigp', 'f1', 'f1finale', 'abudabhigp', 'formula1', 'formulaone', 'lewishamilton', 'maxverstappen33', 'wtf1', 'maxverstappen', 'verstappen', 'fia', 'hamilton', 'skyf1', 'redbull', 'formule1', 'mafia', 'redbullracing', 'historyawaits', 'teamlh', 'mercedesamgf1', 'lh44', 'worldchampion', 'f12021', 'bbcf1', 'finalbattle', 'kimiraikkonen', 'f1naband', 'perez', 'abudhabi', 'mercedes', 'max', 'abudhabigrandprix', 'f1driveroftheday', 'gomax', 'f1jp', 'supermax', 'elevenf1', 'masiout', 'gpabudhabi', 'champion', 'c4f1', 'robbed', 'lewis', 'f1final', 'kiitoskimi', 'ziggosportf1', 'worldchampionships2021', 'checo', 'drivetosurvive', 'driveroftheday', 'saudiarabiagp', 'formel1', 'hamiltonvsverstappen', 'skysportsf1', 'f1abudhabi', 'grandprix', 'maxvslewis', 'mv33', 'sergioperez', 'latifi', 'checoperez', 'kimi', 'michaelmasi', 'unleashthelion', 'goat', 'gpabudabi', 'masi', 'stillwerise', 'deciderinthedesert', 'verstappenvshamilton', 'teammax', 'historymade', 'totowolff', 'legen

In [ ]:
# Keep only top english or relevant hashtags 
tags_to_keep = ['mafia', 'worldchampion', 'robbed', 'goat', 'legend', 'joke', 
                'corrupt', 'farce', 'f1champion', 'karma', 'cheats', 'rigged', 
                'cheating', 'disgraceful', 'shame', 'cheat', 'robbery', 'fix', 
                'disgrace', 'bullshit', 'fail', 'disgusting', 'corruption', 
                'champ', 'worldchampion2021', 'shameful', 'nevergiveup', 
                'shambles', 'blessed']

hashtag_exempt = '|'.join([x for x in tags_to_keep])

hashtag_remove_regex = '(?i)(#(?!' + hashtag_exempt + ')[A-Za-z0-9_]+(?![A-Za-z0-9_]))'
hashtag_remove_regex

'(?i)(#(?!mafia|worldchampion|robbed|goat|legend|joke|corrupt|farce|f1champion|karma|cheats|rigged|cheating|disgraceful|shame|cheat|robbery|fix|disgrace|bullshit|fail|disgusting|corruption|champ|worldchampion2021|shameful|nevergiveup|shambles|blessed)[A-Za-z0-9_]+(?![A-Za-z0-9_]))'

In [ ]:
# Handles to retain
handles_to_keep = ['@Max33Verstappen', '@F1', '@LewisHamilton', '@MercedesAMGF1', 
                   '@redbullracing', '@fia', '@SChecoPerez', 
                   '@GeorgeRussell63', '@HondaRacingF1', '@ValtteriBottas', 
                   '@amgmotorsport', '@redbull', '@Carlossainz55', 
                   '@ScuderiaFerrari', '@WilliamsRacing', 
                   '@alfaromeoracing', '@LandoNorris',  '@NicholasLatifi', 
                    # involvement in incidents (include misspelled accounts)
                   '@Charles_Leclerc', '@SchumacherMick', '@MickSchumacher', 
                   '@Anto_Giovinazzi', '@KimiRaikkonen']

handles_to_keep = set([x.lower() for x in handles_to_keep])

# Append a negative lookahead to each handle 
# to ensure concatenations of these handles do not remain
handles_to_keep = [x + '(?![A-Za-z0-9_])' for x in handles_to_keep]

print(f"{len(handles_to_keep)} top or relevant handles to be kept.")

# For negative lookahead exceptions
exceptions = '|'.join([re.sub('@', '', x) for x in handles_to_keep])

# Find all handles (except those in the negative lookahead) 
handles_remove_regex = '(?i)(@(?!' + exceptions + ')[A-Za-z0-9_]+(?![A-Za-z0-9_]))'
handles_remove_regex

23 top or relevant handles to be kept.


'(?i)(@(?!nicholaslatifi(?![A-Za-z0-9_])|schumachermick(?![A-Za-z0-9_])|carlossainz55(?![A-Za-z0-9_])|landonorris(?![A-Za-z0-9_])|redbullracing(?![A-Za-z0-9_])|georgerussell63(?![A-Za-z0-9_])|max33verstappen(?![A-Za-z0-9_])|scuderiaferrari(?![A-Za-z0-9_])|mercedesamgf1(?![A-Za-z0-9_])|mickschumacher(?![A-Za-z0-9_])|hondaracingf1(?![A-Za-z0-9_])|f1(?![A-Za-z0-9_])|fia(?![A-Za-z0-9_])|schecoperez(?![A-Za-z0-9_])|kimiraikkonen(?![A-Za-z0-9_])|alfaromeoracing(?![A-Za-z0-9_])|williamsracing(?![A-Za-z0-9_])|valtteribottas(?![A-Za-z0-9_])|charles_leclerc(?![A-Za-z0-9_])|anto_giovinazzi(?![A-Za-z0-9_])|amgmotorsport(?![A-Za-z0-9_])|lewishamilton(?![A-Za-z0-9_])|redbull(?![A-Za-z0-9_]))[A-Za-z0-9_]+(?![A-Za-z0-9_]))'

### Regex cleaning

In [ ]:
## Regex removal
# t.co shortened links 
# using this instead of a more general url regex to prevent catching sentences without a space after period
race_df['tweet_clean'] = [re.sub('(https*://)?t\.co/\w+', '', x) for x in race_df['tweet']] 
# Hashtags
race_df['tweet_clean'] = [re.sub(hashtag_remove_regex, '', x) for x in race_df['tweet_clean']]
# Handles
race_df['tweet_clean'] = [re.sub(handles_remove_regex, '', x) for x in race_df['tweet_clean']]
# < 
race_df['tweet_clean'] = [re.sub('&lt;', '<', x) for x in race_df['tweet_clean']]
# > 
race_df['tweet_clean'] = [re.sub('&gt;', '>', x) for x in race_df['tweet_clean']]
# Excess spaces
race_df['tweet_clean'] = [re.sub(' +', ' ', x) for x in race_df['tweet_clean']]
# Leading/trailing whitespaces
race_df['tweet_clean'] = [x.strip() for x in race_df['tweet_clean']] 

In [ ]:
# Number of handles remaining in tweets
pd.DataFrame([len(x) for x in [re.findall('@[A-Za-z0-9]+(?![A-Za-z0-9_])', x) for x in race_df['tweet_clean']]]).value_counts()

0    170560
1     54659
2     21160
3      4855
4      1094
5       243
6        46
7        15
8         2
dtype: int64

In [ ]:
# Check for nan/non-str tweets
i = 0
for x in race_df['tweet_clean']:
  i += 1
  if type(x) != str:
    print(i, x)
  else:
    continue

### Identify junk tweets

In [ ]:
# Identify DOTD voting tweets and some junk tweets I chanced upon 
# Remove before any formal analyses
race_df['junk'] = [bool(re.findall('(?i)(live\s*stream|WATCH LIVE|jamhuri|espn2|(Make your vote here))', x)) for x in race_df['tweet']]
print(f"{len(race_df[race_df['junk'] == True])} potential junk tweets found.")
race_df[race_df['junk'] == True][['tweet', 'hashtags', 'junk']].sample(10)

438 potential junk tweets found.


,tweet,hashtags,junk
80406,#Formula1 live stream https://t.co/klovOUjwsM,[formula1],True
36994,;;;-:::: Formula 1 Streaming F1 live stream ...,"[f1, abudabhigp, abudhabigp, formula1]",True
67572,I voted for S. PEREZ as #F1DriveroftheDay. Mak...,"[f1driveroftheday, abudhabigp]",True
73141,I voted for S. PEREZ as #F1DriveroftheDay. #E...,"[f1driveroftheday, elevenf1, f1pl, abudhabigp]",True
69869,I voted for S. PEREZ as #F1DriveroftheDay. Mak...,[f1driveroftheday],True
203416,Stopped watching #F1 after #Schumacher retired...,"[f1, schumacher, twitter, abudhabigp]",True
75320,I voted for L. HAMILTON as #F1DriveroftheDay. ...,[f1driveroftheday],True
42590,Watch F1 Live from Abu Dhabi – Stream UK a NO...,"[abudhabigp, redbull]",True
1269,Abu Dhabi Grand Prix live stream: How to watch...,[abudabhigp],True
45100,Formel 1 2021 live: Abu Dhabi F1 GP Grand Pri...,"[abudhabigp, redbull]",True


### Output

In [ ]:
print(f"{len(race_df)} tweets.")
race_df[['id', 'date', 'interval', 'tweet', 'tweet_clean', 'hashtags', 'tweet_len', 'junk']]\
.to_parquet('drive/My Drive/data/abudhabigp_tweets_clean.gzip', compression = 'gzip')

252634 tweets.
